In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

/Volumes/Samsung-SSD-T7/Qredence/GraphFleet/GraphFleet/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]

llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [7]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../graphfleet/output/20240828-113421/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [8]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 263
Report count after filtering by community level 2: 208


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,149,# LATS and its Impact on AI and ML Benchmarks\...,2,9.0,LATS and its Impact on AI and ML Benchmarks,The report provides a comprehensive and insigh...,The community revolves around the LATS (Langua...,[{'explanation': 'LATS (Language Agent Tree Se...,"{\n ""title"": ""LATS and its Impact on AI and...",7df98cc7-daf6-401b-9dcf-ae5c8f5f607c
1,150,"# Search Algorithms in AI and ML: A*, DFS, and...",2,8.5,"Search Algorithms in AI and ML: A*, DFS, and LATS",The rating is high due to the significant impa...,The community revolves around the study and ap...,[{'explanation': 'A* is a search algorithm tha...,"{\n ""title"": ""Search Algorithms in AI and M...",db392895-4869-439a-b953-337446d24da6
2,151,# LATS Algorithm and Its Action Space\n\nThe c...,2,8.5,LATS Algorithm and Its Action Space,The rating is high due to the detailed insight...,The community revolves around the LATS algorit...,[{'explanation': 'The LATS algorithm is a pivo...,"{\n ""title"": ""LATS Algorithm and Its Action...",f6f9c187-6d63-4775-a0ff-9b42259b3235
3,152,# Monte Carlo Tree Search (MCTS) and its Role ...,2,9.0,Monte Carlo Tree Search (MCTS) and its Role in...,The rating is high due to the significant impa...,The community revolves around Monte Carlo Tree...,[{'explanation': 'Monte Carlo Tree Search (MCT...,"{\n ""title"": ""Monte Carlo Tree Search (MCTS...",7eed378a-8347-4466-9180-88d87cbfb6bc
4,153,# Sampling and Stochastic Nature in Language M...,2,8.5,Sampling and Stochastic Nature in Language Mod...,The rating is high due to the critical role th...,The community focuses on the interplay between...,[{'explanation': 'Sampling is a crucial proces...,"{\n ""title"": ""Sampling and Stochastic Natur...",24afb751-6e1d-4143-8b3d-ef0e3768ef2e


#### Build global context based on community reports

In [9]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [10]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [11]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=True,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [12]:
result = await search_engine.asearch(
    "Write an example of multi-agent system that incorporates the tree search algorithm? and GraphRAG?Show me an agent that can help me create a knowledge graph"
)

print(result.response)

### Multi-Agent System Incorporating Tree Search Algorithm and GraphRAG

A multi-agent system that incorporates both the tree search algorithm and GraphRAG can be designed to perform complex data retrieval, reasoning, and knowledge graph construction tasks. Below is an example of such a system:

#### System Overview

1. **Data Retrieval Agent**: This agent uses a tree search algorithm, such as Depth-First Search (DFS) or Breadth-First Search (BFS), to explore and retrieve relevant data from a large dataset. The tree search algorithm helps in efficiently navigating through the data to find the most pertinent information [Data: Reports (217, 231, 92)].

2. **Data Processing Agent**: Once the data is retrieved, this agent processes the data to ensure it is clean and structured. It may use various data preprocessing techniques to handle missing values, normalize data, and remove duplicates.

3. **Knowledge Graph Construction Agent**: This agent employs GraphRAG (Graph-based Retrieval-Augme

In [13]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,169,MGSM and Verified Multimodal Agent in AI and ML,0.307692,# MGSM and Verified Multimodal Agent in AI and...,9.0
1,239,META AGENT and Advanced Driver Assistance Syst...,0.307692,# META AGENT and Advanced Driver Assistance Sy...,9.0
2,55,Search and Lookup Actions in HotPotQA and Webshop,0.230769,# Search and Lookup Actions in HotPotQA and We...,8.5
3,142,Microsoft and AI/ML Research Community,0.192308,# Microsoft and AI/ML Research Community\n\nTh...,9.0
4,125,FM_Module and TaskInfo in AI and ML,0.192308,# FM_Module and TaskInfo in AI and ML\n\nThe c...,9.0
...,...,...,...,...,...
203,230,Hierarchical Clustering and Community Structur...,0.076923,# Hierarchical Clustering and Community Struct...,8.5
204,138,FunSearch and Foundation Models,0.076923,# FunSearch and Foundation Models\n\nThe commu...,8.5
205,156,LATS and UCT Algorithm in AI and ML,0.076923,# LATS and UCT Algorithm in AI and ML\n\nThe c...,8.5
206,91,Hierarchical Index in AI and ML,0.038462,# Hierarchical Index in AI and ML\n\nThe commu...,8.5


In [14]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 14. LLM tokens: 161803
